# Fourier series 3 different ways:

In [2]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi, cos, sin
from scipy.signal import correlate, square

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Fourier series of an odd square wave:

$$x(t) = 
\begin{cases} 
  -1 & -\frac{T}{2} < t < 0 \\
  1 & 0 \leq t < \frac{T}{2} 
\end{cases} 
\ \ \ \ \ \ \ \ \
x(t+nT) = x(t)$$

We found that:

$$x(t) \sim \frac{1}{2} \sum_{m=-\infty}^{\infty} S_m e^{i\left(\frac{m\pi}{T} \right) t}$$

$$S_m = \frac{i}{m\pi} \left[ 1 - \cos\left(m\pi\right)\right] =
\begin{cases} 
  \frac{2i}{m\pi} & \text{for } m \text{ odd} \\
  0 & \text{for } m \text{ even}
\end{cases}$$

We can reduce this into a Fourier sine series, which makes calulations easier:

$$x(t) \sim 2\sum_{n=1}^{\infty} b_m \sin\left(\frac{m\pi}{T} t \right)$$

$$b_m = \frac{1}{m\pi} \left[ 1 - \cos\left(m\pi\right)\right] =\frac{1}{m\pi} \left[ 1 - (-1)^m\right]
\begin{cases} 
  \frac{2}{m\pi} & \text{for } m \text{ odd} \\
  0 & \text{for } m \text{ even}
\end{cases}$$

In [3]:
T = np.linspace(-3/2, 3/2, 10000)
M = [1, 2, 5, 10, 100]

## 1) For loops:

In [7]:
def x(m, t):
    x = 0
    for n in range(1, m+1):
        b = 1/(n*pi) * (1 - (-1)**n)
        s = sin(n*pi*t)
        x += 2*b*s 
    return x 

In [6]:
plt.figure(figsize = (10, 4))
for i in range(len(M)):
    plt.plot(T, x(M[i], T), label='n = {0}'.format(M[i]))

plt.plot(T, square(pi*T), label='Exact')
plt.title('Fourier series approximation of an odd square-wave function for $n$ terms and the exact function')
plt.xlabel('$T$')
plt.legend()

<Figure size 720x288 with 1 Axes>

## 2) List comprehension 

In [8]:
series = lambda n, t: 2/(n*pi) * (1 - (-1)**n)*sin(n*pi*t) 

# Create an array with 
# Time values (t) along row
# Term values (n) along column
X = np.array([[series(n, T[i]) for n in range(1, max(M)+1)] for i in range(0, len(T))])

In [57]:
# We need to sum the rows to get the series 
# up to the n^th value
plt.figure(figsize = (10, 4))

# Sums the column elements 
for i in range(len(M)):
    S = 0
    for n in range(0, max(M)):
        S += X[:, n]
        # Plots the columns (n) we care about 
        if n == M[i]-1:
            plt.plot(T, S, label='n = {0}'.format(n+1))

plt.plot(T, square(pi*T), label='Exact')
plt.title('Fourier series approximation of an odd square-wave function for $n$ terms and the exact function')
plt.xlabel('$T$')
plt.legend()

<Figure size 720x288 with 1 Axes>

In [58]:
# Attempt to elliminate the for loop
# Via cumulative sum of column elements 
SL = np.cumsum(X, axis=1)

In [59]:
plt.figure(figsize = (10, 4))

# Pick out the sum we want and plot it 
for i in range(0, len(M)):
    plt.plot(T, SL[:, M[i]-1], label='n = {0}'.format(M[i]))

plt.plot(T, square(pi*T), label='Exact')
plt.title('Fourier series approximation of an odd square-wave function for $n$ terms and the exact function')
plt.xlabel('$T$')
plt.legend()

<Figure size 720x288 with 1 Axes>

## 3) Vectorize

In [116]:
n_terms = 100
steps = 10000
half_interval = 1.5

series = lambda i, j: 2/((j+1)*pi) * (1 - (-1)**(j+1)) * sin((j+1) * pi * (-half_interval + 2*half_interval*i/steps)) 

In [117]:
# Make an array with 
# n terms along the columns 
# t terms along the rows 

XV = np.fromfunction(series, (steps, n_terms))

# Make a cumulative sum of the columms 
SLV = np.cumsum(XV, axis=1)

In [118]:
plt.figure(figsize = (10, 4))

# Pick out the sum we want and plot it 
for i in range(0, len(M)):
    plt.plot(T, SLV[:, M[i]-1], label='n = {0}'.format(M[i]))

plt.plot(T, square(pi*T), label='Exact')
plt.title('Fourier series approximation of an odd square-wave function for $n$ terms and the exact function')
plt.xlabel('$T$')
plt.legend()

<Figure size 720x288 with 1 Axes>